<a href="https://colab.research.google.com/github/javier-fraga-garcia/pagespeed-auditor/blob/main/notebooks/PageSpeed_Auditor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Auditor de Page Speed API

Este cuaderno pretende ser una interfaz de usuario para la realización de auditorías de velocidad de forma automatizada. El objetivo es poder llevar a cabo una gran cantidad de auditorías y que la interacción con el programa sea sencilla. Para ello se ha creado una conexión con la **API de PageSpeed Insights** que permite la realización de auditorías de los **Core Web Vitals** de forma gratuita. Solamente se necesita una clave de API válida y un archivo con las URLs que se desean auditar. En la sección de configuración del entorno se explican los parámetros que es necesario configurar y se proporciona una interfaz para ello. A partir de ahí solo es necesario seguir las instrucciones del cuaderno.

Para más información, o acceso al script de consola de comandos, se puede acceder a este repositorio de [Github](https://github.com/javier-fraga-garcia/pagespeed-auditor).

### Configuración del entorno

In [ ]:
#@markdown #### Configuración de los parámetros para llevar a cabo la auditoría
#@markdown Se deben configurar los siguientes parámetros:
#@markdown * `urls_file` &rarr; la ruta al archivo del que se obtendrán las URLs a auditar. Se recomienda utilizar el gestor de archivos de **Colab** para subir el archivo y acceder a él. El archivo debe contener las URLs en líneas distintas. Para ver un ejemplo puede acceder al repositorio de [Github](https://github.com/javier-fraga-garcia/pagespeed-auditor) y ver la carpeta `examples`.
#@markdown * `domain` &rarr; la dirección dentro de la cual se quieren realizar las auditorías, por ejemplo `https://www.example.com`. El archivo con las URLs puede contener páginas que no interese auditar porque se ha obtenido mediante web scraping o de alguna fuente que ha retornado URLs que apuntan a sitios que no queremos auditar.
#@markdown * `api_key_file` &rarr; la ruta al archivo con la clave de API para conectarse al servicio de Google PageSpeed. El archivo debe ser en formato de texto plano y solo contener la clave de API. Para ver un ejemplo puede acceder al repositorio de [Github](https://github.com/javier-fraga-garcia/pagespeed-auditor) y ver la carpeta `examples`. Para más información sobre como obtener una clave API consultar la documentación de [Google](https://developers.google.com/speed/docs/insights/v5/get-started#APIKey). 
#@markdown * `res_file_name` &rarr; el nombre del archivo de resultados sin la extensión del archivo, por ejemplo `results-mobile`. Cuando la auditoría haya terminado se generará un archivo `csv` con este nombre en el sistema de archivos de **Colab**.
#@markdown * `strategy` &rarr; el dispositivo en el que desea realizar la auditoría (`mobile`, `desktop`). 

#@markdown ---
urls_file = '/content/urls.txt' #@param {type: 'string'}
domain = 'https://www.example.com' #@param {type: 'string'}
api_key_file = '/content/token.txt' #@param {type: 'string'}
res_file_name = 'results' #@param {type: 'string'}
strategy = 'desktop' #@param ['desktop', 'mobile']

### Setup
Ejecutar las celdas siguientes para configurar el entorno

In [ ]:
import concurrent.futures as fs
import requests
import random
import re
import csv

In [ ]:
def get_urls(urls_file, domain):
  try:
    with open(urls_file, 'r') as f:
      return {url.strip() for url in f.read().split('\n') if len(url) > 0 and url.startswith(domain)}
  except:
    print(f'[!] File {urls_file} could not be opened')

def get_api_key(api_key_file):
  try:
    with open(api_key_file, 'r') as f:
      return f.read().strip()
  except:
    print(f'[!] File {api_key_file} could not be opened')

def parse_audit(audit):
  return {
      'final_url': audit['finalUrl'],
      'device': audit['configSettings']['emulatedFormFactor'],
      'first_contentful_paint': float(re.sub('[a-zA-Z_,]', '', audit['audits']['first-contentful-paint']['displayValue']).strip()) if audit['audits']['first-contentful-paint']['displayValue'] else None,
      'time_to_interactive': float(re.sub('[a-zA-Z_,]', '', audit['audits']['first-contentful-paint']['displayValue']).strip()) if audit['audits']['first-contentful-paint']['displayValue'] else None,
      'largest_contentful_paint': float(re.sub('[a-zA-Z_,]', '', audit['audits']['interactive']['displayValue']).strip()) if audit['audits']['interactive']['displayValue'] else None,
      'speed_index': float(re.sub('[a-zA-Z_,]', '', audit['audits']['speed-index']['displayValue']).strip()) if audit['audits']['speed-index']['displayValue'] else None,
      'total_blocking_time': float(re.sub('[a-zA-Z_,]', '', audit['audits']['total-blocking-time']['displayValue']).strip()) if audit['audits']['total-blocking-time']['displayValue'] else None,
      'dom_size': int(re.sub('[a-zA-Z_,]', '', audit['audits']['dom-size']['displayValue']).strip()) if audit['audits']['dom-size']['displayValue'] else None,
      'performance': audit['categories']['performance']['score'] if audit['categories']['performance']['score'] else None,
      'accessibility': audit['categories']['accessibility']['score'] if audit['categories']['accessibility']['score'] else None,
      'best_practices': audit['categories']['best-practices']['score'] if audit['categories']['best-practices']['score'] else None,
      'seo': audit['categories']['seo']['score'] if audit['categories']['seo']['score'] else None
  }

def make_audit(url, api_key, strategy):
  ENDPOINT = 'https://www.googleapis.com/pagespeedonline/v5/runPagespeed?url={}&category=best-practices&category=performance&category=seo&category=accessibility&key={}'
  params = {
      'strategy': strategy
  }
  try:
    print(f'[+] Auditing {url} with strategy {strategy}')
    response = requests.get(ENDPOINT.format(url, api_key), params)
    if response.status_code == 200:
      audit = response.json()['lighthouseResult']
      return parse_audit(audit)
  except:
    print(f'[!] Something went wrong with {url}')

def make_audits(urls, api_key, strategy):
  print(f'[+] Auditing {len(urls)} unique URLs\n')
  audits = list()
  for i, url in enumerate(urls):
    audit = make_audit(url, api_key, strategy)
    audits.append(audit)
    if (i+1) % 25 == 0:
      print(f'\n[-] Completed {i+1} audits\n')
  return audits

def make_concurrent_audits(urls, api_key, strategy, max_workers=5):
  print(f'[+] Auditing {len(urls)} unique URLs\n')
  with fs.ThreadPoolExecutor(max_workers=max_workers) as exec:
    futures = [exec.submit(make_audit, url, api_key, strategy) for url in urls]
    audits = [future.result() for future in fs.as_completed(futures)]
  return audits

def write_results(audits, file_name):
  try:
    with open(f'{file_name}.csv', 'w+', newline='') as f:
      writer = csv.writer(f)
      headers = list(audits[0].keys())
      writer.writerow(headers)

      for audit in audits:
        res = [audit.get(key, None) for key in headers]
        writer.writerow(res)
      print(f'[+] Written file in {file_name}.csv\n')
  except:
    print(f'Something went wrong when writing the file {file_name}')

In [ ]:
urls = get_urls(urls_file, domain)
api_key = get_api_key(api_key_file)

### PageSpeed

#### Muestreo

Desplegar esta sección en caso de querer aplicar muestreo.

In [ ]:
#@markdown Ejecutar la celda para saber el número de URLs únicas a auditar
print(f'There are {len(urls)} unique URLs to audit')

In [ ]:
#@markdown Si no se desea auditar todas las urls puede indicar se puede realizar un muestreo sobre ellas para obtener un subconjunto. Indique los valores correspondientes en las siguientes variables en caso de querer aplicar esta operación y ejecute la celda.
#@markdown * `seed` (opcional) &rarr; este parámetro puede definirse como un número para garantizar la reproducibilidad del resultado, es decir, que siempre que se ejecute sobre el conjunto de URLs se seleccionan las mismas. Por defecto no se aplica.
#@markdown * `n_sample` &rarr; el número de URLs que se desean conservar para la auditoría.

#@markdown ---
seed = 42 #@param {type: 'integer'}
n_sample = 5 #@param {type: 'integer'}

if seed > 0 and n_sample > 0 and n_sample < len(urls):
  random.seed(seed)
  urls = random.sample(urls, n_sample)
  print(f'Generated sample of {n_sample} with seed {seed}')
elif n_sample > 0 and n_sample < len(urls):
  urls = random.sample(urls, n_sample)
  print(f'Generated sample of {n_sample}')
else:
  print('Sampling has not been applied')

#### Auditorías secuenciales

In [ ]:
#@markdown Ejecutar esta celda para comenzar las auditorías. Se irá generando una traza para seguir el proceso. 

#@markdown Al final se reportará también información sobre el tiempo de ejecución y se generará un archivo con los resultados con el nombre indicado en la sección inicial.
%%time
audits = make_audits(urls, api_key, strategy)
write_results(audits, res_file_name)

#### Auditorías multi-hilo

El código siguiente permite una paralelización de las solicitudes a la API. Esto permite mejorar los tiempos de ejecución y reducir el tiempo que se tarda en realizar las auditorías. Es importante tener en cuenta que si se realizan demasiadas peticiones puede que la API retorne un error o que se supere el límite de cuota.

In [ ]:
#@markdown Ejecutar la siguiente celda para ejecutar las peticiones paralelizadas. 

#@markdown *NOTA: solo se recomienda en casos con muchas URLs a auditar y cuya ejecución se demore mucho*

%%time
audits = make_concurrent_audits(urls, api_key, strategy)
write_results(audits, res_file_name)